In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={"lines.linewidth": 2}, palette  = "deep", style = "ticks")
from itertools import product, permutations, combinations, combinations_with_replacement
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import matplotlib.pyplot as plt

In [131]:
truepath_1 = '/gpfs/gibbs/pi/zhao/zc354/GRN/data/Non-specific-ChIP-seq1-GRN.csv'
truepath_2 = '/gpfs/gibbs/pi/zhao/yw599/GRN/BEELINE-Networks/Networks/human/Non-specific-ChIP-seq-network.csv'
predpath_1 = '/gpfs/gibbs/pi/zhao/zc354/GRN/output/TF_GRN_withoutATAC_norm.csv'
predpath_2 =  '/gpfs/gibbs/pi/zhao/zc354/GRN/data/TF_original排序_9.csv'
directed = True
save_path = '/gpfs/gibbs/pi/zhao/zc354/GRN/output'
save_name = '/AUPRC_TF_GRN_withoutATAC_norm_indirected'

In [111]:
trueEdgesDF_1 = pd.read_csv(truepath_1,sep = '\t', header = 0, index_col = None)

In [112]:
trueEdgesDF_1

,Gene1,Gene2
0,AR,PTGS2
1,ARID3A,CDK1
2,ARID3A,CR2
3,ATF2,PTGS2
4,ATF4,CXCL8
...,...,...
3665,ZNF263,NFASC
3666,ZNF263,PACSIN1
3667,ZNF263,ZBTB16
3668,ZNF263,ZFYVE28


In [113]:
trueEdgesDF_1 = trueEdgesDF_1.loc[(trueEdgesDF_1['Gene1'] != trueEdgesDF_1['Gene2'])]

In [114]:
trueEdgesDF_1.drop_duplicates(keep = 'first', inplace=True)

In [115]:
trueEdgesDF_1

,Gene1,Gene2
0,AR,PTGS2
1,ARID3A,CDK1
2,ARID3A,CR2
3,ATF2,PTGS2
4,ATF4,CXCL8
...,...,...
3665,ZNF263,NFASC
3666,ZNF263,PACSIN1
3667,ZNF263,ZBTB16
3668,ZNF263,ZFYVE28


In [132]:
trueEdgesDF_2 = pd.read_csv(truepath_2,sep = ',', header = 0, index_col = None)
predEdgesDF_1 = pd.read_csv(predpath_1, sep = ',', header =  0, index_col = None)
predEdgesDF_2 = pd.read_csv(predpath_2, sep = ',', header =  0, index_col = None)

In [133]:
len(set(predEdgesDF_2['Gene2']))

748

In [117]:
#Drop index in predEdgesDF that Edgeweight=0
predEdgesDF_1.drop(predEdgesDF_1[predEdgesDF_1['Edgeweight'] == 0].index,inplace=True)
predEdgesDF_2.drop(predEdgesDF_2[predEdgesDF_2['Edgeweight'] == 0].index,inplace=True)

In [121]:
predEdgesDF_1

,Gene1,Gene2,Edgeweight
0,MAF,TNFRSF4,0.022684
1,RORA,TNFRSF4,0.018716
2,RORC,TNFRSF4,0.013738
3,GATA3,TNFRSF4,0.009788
4,KLF6,TNFRSF4,0.009379
...,...,...,...
431451,ZSCAN22,CALY,0.000000
431452,ZSCAN23,CALY,0.000000
431453,ZSCAN29,CALY,0.000000
431454,ZSCAN31,CALY,0.000000


In [122]:
predEdgesDF_1.loc[(predEdgesDF_1['Gene2'] == 'TNFRSF4')]

,Gene1,Gene2,Edgeweight
0,MAF,TNFRSF4,0.022684
1,RORA,TNFRSF4,0.018716
2,RORC,TNFRSF4,0.013738
3,GATA3,TNFRSF4,0.009788
4,KLF6,TNFRSF4,0.009379
...,...,...,...
771,ZFY,TNFRSF4,0.000000
772,ZNF205,TNFRSF4,0.000000
773,ZNF662,TNFRSF4,0.000000
774,ZNF704,TNFRSF4,0.000000


In [58]:
predEdgesDF_2

,Gene1,Gene2,Edgeweight
0,TCF4,PDZRN3,2.026733e-01
1,PAX5,BANK1,1.770214e-01
2,XBP1,TXNDC5,1.591947e-01
3,TCF4,SCN9A,1.586981e-01
4,IRF8,ARPP21,1.568413e-01
...,...,...,...
499902,HNF4G,JCHAIN,4.924148e-10
499903,MESP2,JCHAIN,4.924148e-10
499904,ETV1,JCHAIN,2.597353e-10
499905,NR1H4,JCHAIN,2.462074e-10


In [59]:
sum(predEdgesDF_1['Edgeweight'])

555.9999999999826

In [60]:
len(set(predEdgesDF_2['Gene1']))

776

In [61]:
len(set(predEdgesDF_2['Gene2']))

748

In [63]:
len(set(predEdgesDF_2['Gene2'])-set(predEdgesDF_1['Gene2']))

192

In [123]:
pred_2 = predEdgesDF_2.loc[(predEdgesDF_2['Gene2'] == 'TNFRSF4')]

In [124]:
pred_2

,Gene1,Gene2,Edgeweight
1416,RORA,TNFRSF4,2.648612e-02
1855,MAF,TNFRSF4,2.265731e-02
3475,RORC,TNFRSF4,1.573893e-02
7865,RARG,TNFRSF4,9.428040e-03
9265,GATA3,TNFRSF4,8.499584e-03
...,...,...,...
498788,DPF1,TNFRSF4,2.077930e-07
498789,ETV3L,TNFRSF4,2.077930e-07
498790,HOXA3,TNFRSF4,2.077930e-07
498791,NEUROD2,TNFRSF4,2.077930e-07


In [125]:
pred_2.reset_index(inplace = True)

In [126]:
pred_1.reset_index(inplace = True)

In [127]:
pred_1

,level_0,index,Gene1,Gene2,Edgeweight
0,0,0,RORA,TNFRSF4,2.853272e-02
1,1,1,MAF,TNFRSF4,2.348447e-02
2,2,2,RORC,TNFRSF4,1.560608e-02
3,3,3,RARG,TNFRSF4,1.044906e-02
4,4,4,GATA3,TNFRSF4,8.548115e-03
...,...,...,...,...,...
747,747,747,TFCP2L1,TNFRSF4,4.749026e-07
748,748,748,TEAD3,TNFRSF4,3.693687e-07
749,749,749,DLX4,TNFRSF4,2.216212e-07
750,750,750,PKNOX2,TNFRSF4,7.387373e-08


In [128]:
outDF = pd.DataFrame(columns=['pred_1','pred_2','Edgeweight_1','Edgeweight_2'])

In [129]:
outDF['pred_1']= pred_1['Gene1']
outDF['pred_2']= pred_2['Gene1']
outDF['Edgeweight_1']= pred_1['Edgeweight']
outDF['Edgeweight_2']= pred_2['Edgeweight']

In [130]:
outDF.loc[1:50,:]

,pred_1,pred_2,Edgeweight_1,Edgeweight_2
1,MAF,MAF,0.023484,0.022657
2,RORC,RORC,0.015606,0.015739
3,RARG,RARG,0.010449,0.009428
4,GATA3,GATA3,0.008548,0.008500
5,KLF6,BCL11B,0.007784,0.008228
6,BCL11B,PRDM1,0.007759,0.007647
7,PRDM1,ETS1,0.007113,0.007555
8,ETS1,RUNX3,0.006865,0.007332
9,FOXP1,KLF6,0.006335,0.007149
10,FOXP3,FOXP1,0.006124,0.006752


In [72]:
pred_1 = predEdgesDF_1.loc[(predEdgesDF_1['Gene2'] == 'TNFRSF4')]

In [79]:
newDF_2 = pred_2.loc[(pred_2.loc['Edgeweight'] >= 1e-6)]

KeyError: 'Edgeweight'

In [44]:
#Drop selfEdges
trueEdgesDF_2 = trueEdgesDF_2.loc[(trueEdgesDF_2['Gene1'] != trueEdgesDF_2['Gene2'])]
predEdgesDF = predEdgesDF.loc[(predEdgesDF['Gene1'] != predEdgesDF['Gene2'])]

In [45]:
#Drop duplicates in trueEdgesDF and predEdgesDF
trueEdgesDF_1.drop_duplicates(keep = 'first', inplace=True)
predEdgesDF.drop_duplicates(keep = 'first',subset=['Gene1','Gene2'],inplace=True)

In [27]:
true_TF = set(trueEdgesDF_2['Gene1'])
pred_TF = set(predEdgesDF['Gene1'])
true_tg = set(trueEdgesDF_2['Gene2'])
pred_tg = set(predEdgesDF['Gene2'])

In [28]:
TFset = true_TF & pred_TF
tgset = true_tg & pred_tg

In [29]:
trueEdgesDF_2 = trueEdgesDF_2[(trueEdgesDF_2['Gene1'].isin(TFset))&(trueEdgesDF_2['Gene2'].isin(tgset))]
predEdgesDF = predEdgesDF[(predEdgesDF['Gene1'].isin(TFset))&(predEdgesDF['Gene2'].isin(tgset))]

In [30]:
trueEdgesDF_2

,Gene1,Gene2
134,AR,PTGS2
165,ARID3A,CDK1
166,ARID3A,CR2
261,ATF2,PTGS2
292,ATF4,CXCL8
...,...,...
386146,ZNF263,NFASC
386157,ZNF263,PACSIN1
386281,ZNF263,ZBTB16
386284,ZNF263,ZFYVE28


In [32]:
DiffDF = pd.concat([trueEdgesDF_1,trueEdgesDF_2,trueEdgesDF_2]).drop_duplicates(keep = False)

In [35]:
DiffDF

,Gene1,Gene2
80,FOXO4,SOD2
81,FOXO4,TNFSF10
1318,FOXO4,ADAM12
1319,FOXO4,ADAMTS6
1320,FOXO4,BIRC3
1321,FOXO4,GAB2
1322,FOXO4,GLRA1
1323,FOXO4,KHDRBS2
1324,FOXO4,LMNA
1325,FOXO4,MAML3


In [36]:
len(DiffDF)

42

In [38]:
"FOXO4" in TFset

False

In [39]:
"MEIS1" in TFset

False

In [40]:
"NFE2L1" in TFset

False